In [ ]:
!pip install kaggle


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d rishavdash/retail-demand-forecasting-dataset

Dataset URL: https://www.kaggle.com/datasets/rishavdash/retail-demand-forecasting-dataset
License(s): ODbL-1.0
100% 1.76M/1.76M [00:00<00:00, 2.57MB/s]
100% 1.76M/1.76M [00:00<00:00, 2.29MB/s]


In [ ]:
from zipfile import ZipFile
dataset='/content/retail-demand-forecasting-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv("Retail_Dataset2.csv")

In [ ]:

df.head(5)

,Product_id,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Open,Promo,StateHoliday,SchoolHoliday,Petrol_price
0,786725,Product_0033,Whse_S,Category_005,01/03/2016,16000,1,0,0,0,91
1,786743,Product_1825,Whse_S,Category_006,01/03/2016,50000,1,0,0,0,85
2,786967,Product_0551,Whse_S,Category_030,01/03/2016,3000,1,0,0,0,85
3,786856,Product_0556,Whse_S,Category_030,01/03/2016,1000,1,0,0,0,93
4,899538,Product_1844,Whse_A,Category_018,01/03/2016,7,1,0,0,0,95


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [ ]:
state_holiday_mapping = {'0': 0, 'a': 1, 'b': 2}
df['StateHoliday'] = df['StateHoliday'].map(state_holiday_mapping)
df['Day_Type'] = df['Day_Type'].astype(int)

In [ ]:


# df["Date"]=pd.to_datetime(df["Date"]) #, format='%d/%m/%Y'

In [ ]:
df2=df.copy()
df2['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')

In [ ]:
df2['Year'] = df['Date'].dt.year
df2['Month'] = df['Date'].dt.month
df2['Day'] = df['Date'].dt.day

In [ ]:
# df2.drop('Date', axis = 1)

In [ ]:
df2.head(5)

,Product_id,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Open,Promo,StateHoliday,SchoolHoliday,Petrol_price,Day_Type,Year,Month,Day
0,786725,Product_0033,Whse_S,Category_005,2016-01-03,16000,1,0,0,0,91,1,2016,1,3
1,786743,Product_1825,Whse_S,Category_006,2016-01-03,50000,1,0,0,0,85,1,2016,1,3
2,786967,Product_0551,Whse_S,Category_030,2016-01-03,3000,1,0,0,0,85,1,2016,1,3
3,786856,Product_0556,Whse_S,Category_030,2016-01-03,1000,1,0,0,0,93,1,2016,1,3
4,899538,Product_1844,Whse_A,Category_018,2016-01-03,7,1,0,0,0,95,1,2016,1,3


In [ ]:
# df1 = pd.get_dummies(df, columns=['Product_Code', 'Warehouse', 'Product_Category'], drop_first=True)

In [ ]:
# df1.head()

In [ ]:
# #1 weekend, 0 weekday
# df2=df.copy()
# df2['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in ['Product_Code', 'Warehouse', 'Product_Category']:
    le = LabelEncoder()
    df2[column] = le.fit_transform(df2[column])
    label_encoders[column] = le

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Day_Type'] = df['Date'].dt.dayofweek.apply(lambda x: '1' if x >= 5 else '0')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

state_holiday_mapping = {'0': 0, 'a': 1, 'b': 2}
df['StateHoliday'] = df['StateHoliday'].map(state_holiday_mapping)
df['Day_Type'] = df['Day_Type'].astype(int)

df1 = df.groupby(['Product_Code', 'Month', 'Year'], as_index=False).agg({
    'Order_Demand': 'sum',
    'Open': 'first',
    'StateHoliday': 'first',
    'SchoolHoliday': 'first',
    'Day_Type': 'first',
    'Petrol_price': 'mean'
})

df1 = df1.reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df1['Petrol_price_normalized_minmax'] = scaler.fit_transform(df1[['Petrol_price']])

# Display the normalized data
# print(df1[['Product_Code', 'Month', 'Year', 'Petrol_price', 'Petrol_price_normalized_minmax']])


In [ ]:
df2.head()

,Product_id,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Open,Promo,StateHoliday,SchoolHoliday,Petrol_price,Day_Type,Year,Month,Day
0,786725,32,3,2,2016-01-03,16000,1,0,0,0,91,1,2016,1,3
1,786743,1772,3,3,2016-01-03,50000,1,0,0,0,85,1,2016,1,3
2,786967,543,3,26,2016-01-03,3000,1,0,0,0,85,1,2016,1,3
3,786856,548,3,26,2016-01-03,1000,1,0,0,0,93,1,2016,1,3
4,899538,1791,0,15,2016-01-03,7,1,0,0,0,95,1,2016,1,3


In [ ]:
unique_classes = {}
for column in ['Product_Code', 'Warehouse', 'Product_Category']:
    unique_classes[column] = df[column].nunique()

for column, count in unique_classes.items():
    print(f"{column}: {count} unique classes")

Product_Code: 2117 unique classes
Warehouse: 4 unique classes
Product_Category: 30 unique classes


In [ ]:
df2.head()

,Product_id,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Open,Promo,StateHoliday,SchoolHoliday,Petrol_price,Day_Type,Year,Month,Day
0,786725,32,3,2,2016-01-03,16000,1,0,0,0,91,1,2016,1,3
1,786743,1772,3,3,2016-01-03,50000,1,0,0,0,85,1,2016,1,3
2,786967,543,3,26,2016-01-03,3000,1,0,0,0,85,1,2016,1,3
3,786856,548,3,26,2016-01-03,1000,1,0,0,0,93,1,2016,1,3
4,899538,1791,0,15,2016-01-03,7,1,0,0,0,95,1,2016,1,3


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

label_encoder_product = LabelEncoder()
df1['Product_Code'] = label_encoder_product.fit_transform(df1['Product_Code'])

label_encoder_stateholiday = LabelEncoder()
df1['StateHoliday'] = label_encoder_stateholiday.fit_transform(df1['StateHoliday'])

features = ['Product_Code', 'Month', 'Open', 'StateHoliday', 'SchoolHoliday', 'Order_Demand']

scaler = MinMaxScaler()
df1[features] = scaler.fit_transform(df1[features])

def create_sequences(data, seq_length=3):
    X, y = [], []
    for i in range(len(data) - seq_length):
        # Input features (current and previous months)
        X.append(data.iloc[i:i + seq_length][features].values.reshape(-1))  # Flatten to 1D
        # The target variable (next month's order demand)
        y.append(data.iloc[i + seq_length]['Order_Demand'])

    return np.array(X), np.array(y)

X_all, y_all = [], []
products = df1['Product_Code'].unique()

for product in products:
    product_data = df1[df1['Product_Code'] == product].sort_values(by=['Year', 'Month'])

    if len(product_data) >= 3:
        X_seq, y_seq = create_sequences(product_data, seq_length=3)
        if X_seq.shape[0] > 0:
            X_all.append(X_seq)
            y_all.append(y_seq)

X = np.concatenate(X_all, axis=0)  # X is 2D: [samples, flattened features]
y = np.concatenate(y_all, axis=0)  # y is 1D: [samples]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")


Mean Absolute Error (MAE): 0.002186216442656605
Mean Squared Error (MSE): 0.00020896375992415133
Root Mean Squared Error (RMSE): 0.014455578851230805
R² Score: 0.7972199381124174


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

label_encoder_product = LabelEncoder()
df1['Product_Code'] = label_encoder_product.fit_transform(df1['Product_Code'])

label_encoder_stateholiday = LabelEncoder()
df1['StateHoliday'] = label_encoder_stateholiday.fit_transform(df1['StateHoliday'])

features = ['Product_Code', 'Month', 'Open', 'StateHoliday', 'SchoolHoliday', 'Order_Demand']

scaler = MinMaxScaler()
df1[features] = scaler.fit_transform(df1[features])

def create_sequences(data, seq_length=3):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length][features].values.reshape(-1))
        y.append(data.iloc[i + seq_length]['Order_Demand'])
    return np.array(X), np.array(y)

X_all, y_all = [], []
products = df1['Product_Code'].unique()

for product in products:
    product_data = df1[df1['Product_Code'] == product].sort_values(by=['Year', 'Month'])
    if len(product_data) >= 3:
        X_seq, y_seq = create_sequences(product_data, seq_length=3)
        if X_seq.shape[0] > 0:
            X_all.append(X_seq)
            y_all.append(y_seq)

X = np.concatenate(X_all, axis=0)
y = np.concatenate(y_all, axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")


Mean Absolute Error (MAE): 0.0023140298820878116
Mean Squared Error (MSE): 0.0002661854176455796
Root Mean Squared Error (RMSE): 0.016315189782701873
R² Score: 0.7416915953113832


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

label_encoder_product = LabelEncoder()
df1['Product_Code'] = label_encoder_product.fit_transform(df1['Product_Code'])

label_encoder_stateholiday = LabelEncoder()
df1['StateHoliday'] = label_encoder_stateholiday.fit_transform(df1['StateHoliday'])


features = ['Product_Code', 'Month', 'Open', 'StateHoliday', 'SchoolHoliday', 'Order_Demand']


scaler = MinMaxScaler()
df1[features] = scaler.fit_transform(df1[features])


def create_sequences(data, seq_length=3):
    X, y = [], []


    for i in range(len(data) - seq_length):

        X.append(data.iloc[i:i + seq_length][features].values)

        y.append(data.iloc[i + seq_length]['Order_Demand'])

    return np.array(X), np.array(y)


X_all, y_all = [], []
products = df1['Product_Code'].unique()

for product in products:
    product_data = df1[df1['Product_Code'] == product].sort_values(by=['Year', 'Month'])
    if len(product_data) >= 3:
        X_seq, y_seq = create_sequences(product_data, seq_length=3)


        if X_seq.shape[0] > 0:
            X_all.append(X_seq)
            y_all.append(y_seq)
X = np.concatenate(X_all, axis=0)  # X is 3D: [samples, time_steps, features]
y = np.concatenate(y_all, axis=0)  # y is 1D: [samples]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])



model.compile(optimizer='adam', loss='mse', metrics=['mae'])


history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test), verbose=1)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


676/676 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0015 - mae: 0.0103 - val_loss: 2.6768e-04 - val_mae: 0.0035
Epoch 2/20
676/676 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 6.0480e-04 - mae: 0.0074 - val_loss: 2.7541e-04 - val_mae: 0.0063
Epoch 3/20
676/676 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 1.9642e-04 - mae: 0.0053 - val_loss: 2.9399e-04 - val_mae: 0.0073
Epoch 4/20
676/676 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 3.0087e-04 - mae: 0.0060 - val_loss: 2.8597e-04 - val_mae: 0.0047
Epoch 5/20
676/676 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 2.2578e-04 - mae: 0.0049 - val_loss: 3.4179e-04 - val_mae: 0.0068
Epoch 6/20
676/676 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 1.8056e-04 - mae: 0.0052 - val_loss: 2.6321e-04 - val_mae: 0.0067
Epoch 7/20
676/676 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.6247e-04 - mae: 0.0053 - val_loss: 2.4985e-04 - val_mae: 0.0039
Epoch 8/20
676/676 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 1.5615e-04 - mae: 0.0045 - val_loss: 2.3267e-04 - val_mae: 0.0037
Epoch 9

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 3, 64)               │          18,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 3, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 95,045 (371.27 KB)

 Trainable params: 31,681 (123.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 63,364 (247.52 KB)